In [68]:
from pandas import DataFrame, Series


def momentum_trading_strategy(data, short_window=50, long_window=200, entry_threshold=0.02, exit_threshold=0.01):
    """
    Parameters:
    - data: DataFrame with 'Date', 'Price', and any additional columns needed for analysis.
    - short_window: Short-term moving average window.
    - long_window: Long-term moving average window.
    - entry_threshold: Minimum price change required to trigger a buy/sell signal (percentage).
    - exit_threshold: Minimum price change required to exit a position (percentage).

    Returns:
    - signals: DataFrame with buy/sell signals (1 for buy, -1 for sell, 0 for hold).
    """

    shorts = data.rolling(window=short_window, min_periods=1).mean()
    longs = data.rolling(window=long_window, min_periods=1).mean()

    signals = [0] * len(data)

    returns = data.pct_change()

    for i in range(1, len(data)):
        if (
            shorts[i] > longs[i] and
            returns[i] > entry_threshold
        ):
            signals[i] = 1  

        elif (
            shorts[i] < longs[i] and
            returns[i] < -entry_threshold
        ):
            signals[i] = -1  

    for i in range(1, len(data)):
        if (
            signals[i] == 1 and
            returns[i] < -exit_threshold
        ):
            signals[i] = 0  

        elif (
            signals[i] == -1 and
            returns[i] > exit_threshold
        ):
            signals[i] = 0  

    return signals



In [79]:
from typing import Callable, List, Tuple, Dict, Any
import yfinance as yf
from pandas import Series, Timestamp
from datetime import datetime, timedelta

class Backtester:
  def __init__(self, strategy: Callable):
    self.strategy = strategy


  def testTickerReport(self, ticker: str, start: str, end: str, startingAmount: float = 1000000.00) -> Dict[str, Any]: # returns a tuple of a dictionary of dates and prices, and a list of returns
    stock = yf.Ticker(ticker)
    data = stock.history(start=start, end=end)
    data = data['Close']
    return self.testCustomReport(data, start, end, startingAmount)

  @staticmethod
  def getRatios(pctChanges: List[float], maxDrawdown: float, overallReturn: float, n: int) -> List[float]:
    pct = Series(pctChanges)

    current_date = Timestamp(datetime.today())
    if current_date.weekday() == 5:  # Saturday
      current_date -= timedelta(days=1)
    elif current_date.weekday() == 6:  # Sunday
      current_date -= timedelta(days=2)
    riskFreeRate = yf.download('^IRX', start=current_date, end=current_date)['Adj Close'][0]/100
    print(riskFreeRate)

    sharpe = (overallReturn-riskFreeRate)/pct.std()
    sortino = (overallReturn-riskFreeRate)/pct[pct < 0].std()
    calmar = (overallReturn-riskFreeRate)/abs(maxDrawdown)
    return [sharpe, sortino, calmar]


  def testCustomReport(self, data: Series, start: str, end: str, startingAmount: float = 1000000.00) -> float:
    signals = self.strategy(data) # Should return list of -1, 0, 1
    amt = startingAmount
    histArr = []
    trades = []
    drawdowns = []
    bought = False
    boughtPrice = 0
    shares = 0
    for price, signal in zip(data, signals):
      if signal == 1 and not bought: # Buying a stock
        bought = True
        boughtPrice = minPrice = price
        currentDuration = 0
        currentDrawdownDuration = 0
        shares = amt // price
        amt -= (shares * price)
      elif signal == -1 and bought: # Selling a stock
        bought = False
        trades.append((currentDuration, (price-boughtPrice)/boughtPrice))
        if currentDrawdownDuration:drawdowns.append((currentDrawdownDuration, (minPrice-price)/price))
        amt += (shares * price)
        shares = 0
      if bought:
        if price < boughtPrice:
          currentDrawdownDuration += 1
        else:
          if currentDrawdownDuration:drawdowns.append((currentDrawdownDuration, (minPrice-price)/price))
          currentDrawdownDuration = 0

        currentDuration += 1
        minPrice = min(minPrice, price)
      histArr.append(shares*price+amt)
    if bought:
      trades.append((currentDuration, (price-boughtPrice)/boughtPrice))
      if currentDrawdownDuration:drawdowns.append((currentDrawdownDuration, (minPrice-price)/price))
    totals = histArr
    
    report = {}
    report['Start'] = start
    report['End'] = end
    report['Duration'] = len(totals)
    report['Exposure Time'] = sum([x[0] for x in trades])/len(trades)
    report['Net Worth'] = totals

    report['Equity Final'] = totals[-1]
    report['Equity Peak'] = max(totals)

    report['Return'] = (totals[-1]-startingAmount)/startingAmount
    report['Buy and Hold Return'] = (data[-1]-data[0])/data[0]

    report['Max Drawdown'] = min([d[1] for d in drawdowns])
    report['Avg Drawdown'] = sum(d[1] for d in drawdowns)/len(drawdowns)
    report["Max Drawdown Duration"] = max(d[0] for d in drawdowns)
    report["Avg Drawdown Duration"] = sum(d[0] for d in drawdowns)/len(drawdowns)
    
    report["# Trades"] = len(trades)
    report["Win Rate"] = sum(1 for t in trades if t[1] > 0)/len(trades)
    report["Best Trade"] = max(t[1] for t in trades)
    report["Worst Trade"] = min(t[1] for t in trades)

    report["Max Trade Duration"] = max(t[0] for t in trades)
    report["Avg Trade Duration"] = sum(t[0] for t in trades)/len(trades)

    report["Sharpe Ratio"], report["Sortino Ratio"], report["Calmar Ratio"] = self.getRatios([x[1] for x in trades], report["Max Drawdown"], report["Return"], len(trades))
    
    return report

  def graphTicker(self, ticker: str, start: str, end: str) -> None:
    pass

  def graphCustom(self, data: List[float]) -> None:
    pass

# How to factor in a look back period for a potential strategy
# Data clean up and organization between different sources

In [80]:
def wrapper(data):
  return momentum_trading_strategy(data, short_window=50, long_window=200, entry_threshold=0.02, exit_threshold=0.01)

bt = Backtester(wrapper)
test = bt.testTickerReport('AAPL', '2010-01-01', '2020-01-01')
for key in test:
  print(key, test[key])

[*********************100%***********************]  1 of 1 completed
0.05320000171661377
Start 2010-01-01
End 2020-01-01
Duration 2516
Exposure Time 470.5
Net Worth [1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1027968.3